In [1]:
import os
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
def assemble_stats(year):
    _dir = os.listdir(year + 'eve')
    _dir = [x for x in _dir if x[0] == '2']

    # make two dfs - one building season stats and the other assembling each game senario

    season_df = pd.DataFrame(columns=['player_id', 'PAs', 'Hits'])
    senario_df = pd.DataFrame()
    game_variables = {}
    starting_pitchers = []
    pitchers = pd.DataFrame()
    
    for file in _dir:
        f = open(year + 'eve/' + file, 'r')
        print(file)
        for line in f:
            line = line.split(',')

            if line[0] == 'id':
                game_variables = {}
                starting_pitchers = []
            if line[0] == 'version':
                pass

            if line[0] == 'info':
                if line[1] == 'visteam':
                    pass
    #                 if line[2].strip() == team:
    #                     home = False
                elif line[1] == 'hometeam':
                    pass
    #                 if line[2].strip() == team:
    #                     home = True

                else:
                    if line[1] not in ['wp', 'lp', 'save']:
                        game_variables[line[1]] = line[2].strip()
            if line[0] == 'start':
                if line[3] == '0' and line[5] == '1\n':
                    starting_pitchers.append(line[1])
                    pitchers.append({'player_id': line[1]})
#                     starting_pitchers['away_SP'] = line[1]
                elif line[3] == '1' and line[5] == '1\n':
                    starting_pitchers.insert(0, line[1])
#                     starting_pitchers['home_SP'] = line[1]

            if line[0] == 'play':
                if line[6] != 'NP\n':

                    if line[3] not in season_df['player_id'].unique():
                        season_df = season_df.append({'player_id': line[3], 'PAs': 0, 'Hits': 0}, ignore_index = True)


                    if line[5][-1] == 'X':

                        if line[6][0] in ['S', 'D', 'T', 'H']:
                            season_df.loc[season_df['player_id'].isin([line[3]]), ['PAs']] += 1
                            season_df.loc[season_df['player_id'].isin([line[3]]), ['Hits']] += 1
                            senario_df = senario_df.append({**game_variables, 
                                                **{'hit': 1, 'player': line[3],
                                                 'PAs': season_df[season_df['player_id'] == line[3]]['PAs'].values[0], 
                                                   'Hits': season_df[season_df['player_id'] == line[3]]['Hits'].values[0],
                                                  'Opposing_SP': starting_pitchers[int(line[2])]}}, 
                                               ignore_index=True)
                        else:
                            season_df.loc[season_df['player_id'].isin([line[3]]), 'PAs'] += 1
                            senario_df = senario_df.append({**game_variables, 
                                                **{'hit': 0, 'player': line[3],
                                                 'PAs': season_df[season_df['player_id'] == line[3]]['PAs'].values[0], 
                                                   'Hits': season_df[season_df['player_id'] == line[3]]['Hits'].values[0],
                                                  'Opposing_SP': starting_pitchers[int(line[2])]}}, 
                                               ignore_index=True)
                            
                    elif line[5][-1] in ['C', 'S', 'B', 'T', 'H', 'M', 'V', 'K', 'U', 'Q'] and line[6][:2] not in ['WP', 'PB', 'SB', 'CS']:
                        season_df.loc[season_df['player_id'].isin([line[3]]), 'PAs'] += 1
                        senario_df = senario_df.append({**game_variables, 
                                            **{'hit': 0, 'player': line[3],
                                             'PAs': season_df[season_df['player_id'] == line[3]]['PAs'].values[0], 
                                               'Hits': season_df[season_df['player_id'] == line[3]]['Hits'].values[0],
                                                  'Opposing_SP': starting_pitchers[int(line[2])]}}, 
                                           ignore_index=True)

        
#         move later in code to allow for manipulation of cumulative stats
#         if not senario_df.empty:
#             senario_df = senario_df.sort_values('hit').drop_duplicates(subset=['date', 'number', 'player'], keep='last')
    return(senario_df, season_df)

In [ ]:
senario_df, season_df = assemble_stats('2019')
senario_df.to_csv('2019eve/senario_df.csv', index=False)
season_df.to_csv('2019eve/season_df.csv', index=False)

senario_df, season_df = assemble_stats('2020')
senario_df.to_csv('2020eve/senario_df.csv', index=False)
season_df.to_csv('2020eve/season_df.csv', index=False)

box score processing


https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2FLAN%2FLAN202107210.shtml&div=div_SanFranciscoGiantsbatting

https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2FCIN%2FCIN202107210.shtml&div=div_NewYorkMetsbatting

https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2FLAN%2FLAN202107210.shtml&div=div_SanFranciscoGiantspitching

https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2F{oppent_abr}%2F{oppent_abr}{date}.shtml&div=div_{team_full_name}{batting/pitching}

In [ ]:
# import schedules
schedule_2019 = pd.read_csv('2019SKED.TXT', columns=['date', 'doubleheader', 'day_of_week', 'visiting_team', 'visiting_league', 'v_game_number', 'home_team', 'home_league', 'h_game_number', 'daynight', 'postponed', 'makeup'])
schedule_2020 = pd.read_csv('2020REV.TXT', columns=['date', 'doubleheader', 'day_of_week', 'visiting_team', 'visiting_league', 'v_game_number', 'home_team', 'home_league', 'h_game_number', 'daynight', 'postponed', 'makeup'])
schedule_2021 = pd.read_csv('2021SKED.TXT', columns=['date', 'doubleheader', 'day_of_week', 'visiting_team', 'visiting_league', 'v_game_number', 'home_team', 'home_league', 'h_game_number', 'daynight', 'postponed', 'makeup'])


In [ ]:
# baseball-reference api

home_team = ''.upper()   # NYM
date = ''                 # 202107210
team_full_name = ''       # NewYorkMets

URL = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2F{home_team}%2F{home_team}{date}.shtml&div=div_{team_full_name}{batting_pitching}'

In [4]:
# Visual Crossing api (weather data)

'''Data for multiple locations can be requested in a single request
by concatenating multiple locations using the pipe (|) character.
'''
'''strat for writing api calls

group by date and time'''
from api_keys import visual_crossing
key = visual_crossing

URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={date}T{start_time}:00&endDateTime={date}T{start_time}:00&unitGroup=us&contentType=csv&location={locations}&key={key}'


In [5]:
import statsapi

statsapi.schedule(date=None, start_date=None, end_date=None, team="", opponent="", sportId=1, game_id=567343)



[{'game_id': 567343,
  'game_datetime': '2019-09-29T19:10:00Z',
  'game_date': '2019-09-29',
  'game_type': 'R',
  'status': 'Final',
  'away_name': 'Detroit Tigers',
  'home_name': 'Chicago White Sox',
  'away_id': 116,
  'home_id': 145,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Ross Detwiler',
  'away_probable_pitcher': 'Spencer Turnbull',
  'home_pitcher_note': "The White Sox will go with a bullpen day for the season finale, with Detwiler set to open. The left-hander held the Tigers to two earned runs over six innings the only time he's seen them this season, July 3.",
  'away_pitcher_note': 'Turnbull starts the season finale looking to build on the momentum of a solid stretch run, having tossed 11 innings of three-run ball with 13 strikeouts over his past two starts. He remains winless since May 31 despite his recent stinginess.',
  'away_score': 3,
  'home_score': 5,
  'current_inning': 9,
  'inning_state': 'Top',
  'venue_id': 4,
  'venue_name': 'Guarant

In [5]:
data = pd.read_csv('2019_statcast_pbp.csv')

In [6]:
data

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,2098,FC,2019-09-29,91.1,-2.16,6.08,"Herrera, Kelvin",641513,516969,strikeout,swinging_strike,NaN,NaN,NaN,NaN,14.0,Travis Demeritte strikes out swinging.,R,R,R,CWS,DET,S,2.0,NaN,2,2,2019,-0.05,0.40,0.08,1.02,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,641470.0,NaN,NaN,5.659540,-132.261173,-7.523466,-1.822484,29.552101,-26.184787,3.34,1.64,NaN,NaN,NaN,89.9,2347.0,5.8,567343,516969,641470.0,572365.0,570560.0,660162.0,664901.0,650391.0,641553.0,641477.0,54.71,NaN,NaN,0.0,1.0,0.0,0.0,NaN,66,5,Cutter,5,3,3,5,3,5,3,5,Standard,Standard,187.0,0.011,-0.099
1,2115,FC,2019-09-29,88.2,-2.22,6.07,"Herrera, Kelvin",641513,516969,NaN,foul,NaN,NaN,NaN,NaN,5.0,Travis Demeritte strikes out swinging.,R,R,R,CWS,DET,S,NaN,NaN,2,1,2019,0.19,0.46,0.16,2.29,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,641470.0,NaN,NaN,5.356255,-128.270061,-3.958404,0.855951,27.381318,-26.670237,3.50,1.64,10.0,66.6,-12.0,87.1,2366.0,5.7,567343,516969,641470.0,572365.0,570560.0,660162.0,664901.0,650391.0,641553.0,641477.0,54.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,4,Cutter,5,3,3,5,3,5,3,5,Standard,Standard,158.0,0.000,-0.033
2,2265,FF,2019-09-29,95.8,-1.55,6.41,"Herrera, Kelvin",641513,516969,NaN,ball,NaN,NaN,NaN,NaN,11.0,Travis Demeritte strikes out swinging.,R,R,R,CWS,DET,B,NaN,NaN,1,1,2019,-0.90,1.42,-1.57,2.78,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,641470.0,NaN,NaN,1.949929,-139.338956,-7.096962,-12.028052,30.328764,-12.541891,3.27,1.64,NaN,NaN,NaN,95.2,2235.0,5.9,567343,516969,641470.0,572365.0,570560.0,660162.0,664901.0,650391.0,641553.0,641477.0,54.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,3,4-Seam Fastball,5,3,3,5,3,5,3,5,Standard,Standard,212.0,0.000,0.024
3,2279,FC,2019-09-29,90.1,-2.16,6.01,"Herrera, Kelvin",641513,516969,NaN,ball,NaN,NaN,NaN,NaN,13.0,Travis Demeritte strikes out swinging.,R,R,R,CWS,DET,B,NaN,NaN,0,1,2019,0.14,0.35,-0.06,0.95,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,641470.0,NaN,NaN,4.867683,-130.869340,-7.192694,0.460072,28.163834,-26.922956,3.34,1.64,NaN,NaN,NaN,88.7,2335.0,5.6,567343,516969,641470.0,572365.0,570560.0,660162.0,664901.0,650391.0,641553.0,641477.0,54.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,2,Cutter,5,3,3,5,3,5,3,5,Standard,Standard,159.0,0.000,0.012
4,2394,FF,2019-09-29,98.2,-1.47,6.24,"Herrera, Kelvin",641513,516969,NaN,called_strike,NaN,NaN,NaN,NaN,5.0,Travis Demeritte strikes out swinging.,R,R,R,CWS,DET,S,NaN,NaN,0,0,2019,-0.34,1.17,-0.21,2.78,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,641470.0,NaN,NaN,4.226199,-142.841106,-6.628930,-5.683070,32.358288,-14.785764,3.48,1.64,NaN,NaN,NaN,98.5,2216.0,6.4,567343,516969,641470.0,572365.0,570560.0,660162.0,664901.0,650391.0,641553.0,641477.0,54.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,1,4-Seam Fastball,5,3,3,5,3,5,3,5,Standard,Standard,196.0,0.000,-0.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [4]:
help("modules")


Please wait a moment while I gather a list of all available modules...



/Users/home/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."
/Users/home/opt/anaconda3/envs/learn-env/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The matplotlib.compat module was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  __import__(info.name)
/Users/home/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/nltk/twitter/__init__.py:21: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn(


Cryptodome          binhex              jupyter             reprlib
Cython              bisect              jupyter_client      requests
IPython             bleach              jupyter_contrib_core requests_html
OpenSSL             boto                jupyter_contrib_nbextensions requests_oauthlib
PIL                 boto3               jupyter_core        resource
__future__          botocore            jupyter_highlight_selected_word retrying
_abc                branca              jupyter_nbextensions_configurator rlcompleter
_ast                brotli              jupyterlab_pygments rmagic
_asyncio            bs4                 jwt                 robin_stocks
_bisect             bson                keras               rsa
_blake2             builtins            keras_preprocessing runpy
_bootlocale         bz2                 keyword             s3transfer
_bz2                bz2file             kiwisolver          sched
_cffi_backend       cProfile            korean_lunar_calen

/Users/home/opt/anaconda3/envs/learn-env/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning:

zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    

